In [130]:
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.io import ascii
import pandas as pd
import os
from astropy.table import Table, Column, join, vstack
import numpy as np
import matplotlib.pyplot as plt
import time
#import seaborn as sns
%matplotlib inline

In [109]:
data=Table.read("/Users/kschles/Documents/GALAH/HVS/update_08112015/sobject_iraf_general.fits")

In [110]:
data2=data.to_pandas()

In [111]:
#Output smaller file to use to query USNO via STILTS and TAP
data2[['sobject_id', 'ra', 'dec']].to_csv('propermotion_temp.csv', index=False)

In [61]:
### Want to find all matches in USNO-B within 7" of the target
os.system('java -jar stilts.jar cdsskymatch cdstable=I/284/out find=all in=propermotion_temp.csv ifmt=csv ra=ra dec=dec radius=7 out=usnob_7as.csv')

0

In [112]:
## Read in the 7" radius query and use it to identify nearest neighbors and matches. 
usnob=pd.read_csv('/Users/kschles/Documents/GALAH/propermotion/usnob_7as.csv')

In [113]:
usnob.loc[np.where(usnob['angDist']<=1.0)[0],'match']=1
usnob.loc[np.where(((usnob['Bmag1']<22.) | (usnob['Bmag2']<22.)) & (usnob['angDist']>1.0))[0],'neighbor']=1

In [114]:
test=usnob.groupby(['sobject_id'])

In [115]:
prelim_quality=test.agg({'sobject_id': 'first', 'match' : np.sum, 'neighbor' : np.sum, 'angDist' : lambda x: min(x)})

In [116]:
prelim_quality['sobject_id']=prelim_quality['sobject_id'].astype(int)

In [117]:
pquality=Table.from_pandas(prelim_quality)

In [121]:
combo1=join(data,pquality[['sobject_id', 'match', 'neighbor']], keys='sobject_id', join_type='left')

In [127]:
ascii.write(data[usno_index[j-1]+1:usno_index[j]][['sobject_id', 'ra', 'dec']], 'test.csv', format='csv')

In [138]:
usno_index=np.append([np.arange(0,len(data),1750)],[len(data)-1])
#for j in range(1,len(usno_index)) :
for j in range(1,10) :
    print usno_index[j-1], time.ctime()
    if (j!=1):
        #data.loc[usno_index[j-1]+1:usno_index[j],['sobject_id', 'ra', 'dec']].to_csv('propermotion_temp.csv', index=False)
        ascii.write(data[usno_index[j-1]+1:usno_index[j]][['sobject_id', 'ra', 'dec']],'propermotion_temp.csv', format='csv')
    if (j==1): 
        #data.loc[usno_index[j-1]:usno_index[j],['sobject_id', 'ra', 'dec']].to_csv('propermotion_temp.csv', index=False)
        ascii.write(data[usno_index[j-1]:usno_index[j]][['sobject_id', 'ra', 'dec']],'propermotion_temp.csv', format='csv')
    temp=os.system('java -jar stilts.jar tapskymatch tapurl=http://dc.g-vo.org/tap taptable=usnob.data in=propermotion_temp.csv ifmt=csv inlon=ra inlat=dec out=usnob_2as_temp.fits taplon=RAJ2000 taplat=DEJ2000 sr=0.00050 find=each') 
    os.system('mv usnob_2as_temp.fits usnob_output/usnob_2as_'+str(j).zfill(3)+'.fits')


0 Sat May  7 15:17:15 2016
1750 Sat May  7 15:17:23 2016
3500 Sat May  7 15:17:32 2016
5250 Sat May  7 15:17:41 2016
7000 Sat May  7 15:17:49 2016
8750 Sat May  7 15:17:57 2016
10500 Sat May  7 15:18:05 2016
12250 Sat May  7 15:18:14 2016
14000 Sat May  7 15:18:22 2016


In [139]:
# Read in and combine the USNO-B files created using TAP
#for j in range(1,len(usno_index)) :
for j in range(1,10) :
    filename='usnob_output/usnob_2as_'+str(j).zfill(3)+'.fits'
    temp=Table.read(filename)
    if j==1: 
        usnob_best=temp
    if j>1: 
        usnob_best=vstack([usnob_best, temp])

In [164]:
combo=join(combo1,usnob_best[['sobject_id', 'fit_ra', 'fit_de', 'ndet', 'sep_arcsec']], \
           keys='sobject_id', join_type='left')

In [157]:
# There should be no rows here! --> ISSUE WITH ISNAN AND NDET WITH ASTROPY TABLE STRUCTURE
combo[np.where((combo['match']>=1) & (np.isnan(combo['ndet'])==True))[0]]['sobject_id', 'ra', 'dec']

sobject_id,ra,dec
,deg,deg
int64,float64,float64


In [166]:
#Adjust the fit_RA and fit_DE values from degrees to mas
combo['fit_ra']=np.round(combo['fit_ra']*3600.*1000.)
combo['fit_de']=np.round(combo['fit_de']*3600.*1000.)
combo['fit_ra'].unit='mas'
combo['fit_de'].unit='mas'

In [169]:
combo.write('sobject_iraf_pmquality.fits')